In [ ]:
# pachages


In [1]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import Ollama


In [8]:
llm = Ollama(
    model="mistral"
)
# callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])

In [9]:
llm('Tell me about the history of Medicine')

"\nMedicine has a long and rich history, dating back thousands of years. Early forms of medicine were practiced by various ancient civilizations, such as the Egyptians, Greeks, Romans, Chinese, and Indians. These early practitioners used a variety of methods to treat illnesses, including herbal remedies, surgery, and magical or spiritual practices.\n\nOne of the earliest known medical texts is the Ebers Papyrus, which dates back to around 1500 BCE and was written in ancient Egyptian. This papyrus contains information on various aspects of medicine, including diagnosis, treatment, and prevention of diseases.\n\nIn ancient Greece, the father of Western medicine is considered to be Hippocrates (460-370 BCE). He is known for his Hippocratic Oath, which outlines the principles of medical ethics. Other notable Greek physicians include Galen (129-216 CE), who made important contributions to the fields of anatomy and pathology, and Asclepius (600-500 BCE), the founder of the first hospitals in

In [10]:
llm('Who is weirdest human alive ')

'\nThere are many individuals who have been deemed "weird" by society, often due to their unique behaviors or beliefs. Some examples include:\n\n* John C. Lilly, a researcher and pioneer in the field of altered states of consciousness who conducted experiments with sensory deprivation tanks\n* Robert Wadlow, also known as the "Alton Giant," was a tall man who measured 8 feet 11 inches at the time of his death at age 22\n* Terence McKenna, a philosopher and psychonaut who has written extensively about altered states of consciousness and psychedelic experiences\n* David Beckham, a former professional soccer player known for his eccentric fashion choices and hairstyles\n* Dwight Schrute, played by Rainn Wilson on the TV show "The Office," is known for his obsession with beets, bears, and Battlestar Galactica.\n\nIt\'s important to note that what may seem weird or unusual to one person may not necessarily be so to another, and it can also depend on cultural context and personal experiences

In [11]:
from langchain.agents import initialize_agent, Tool, AgentType


In [12]:
from langchain.chains import LLMChain
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    PromptTemplate,
)
from langchain.schema import SystemMessage


prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content='you are a very smart AI career advisor'
        ),  # The persistent system prompt
        HumanMessagePromptTemplate.from_template(
            "{human_input}"
        ),  # Where the human input will injectd
    ],
)

conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=False,
)

In [13]:
conversation.predict(human_input='What makes you so unique')

'\nAs a career advisor AI, I have access to vast amounts of information and can analyze data quickly and accurately. This allows me to provide personalized recommendations based on your unique skills, experiences, and goals. Additionally, I am constantly learning from new data and feedback, so I am able to adapt my advice over time and provide the most up-to-date and relevant guidance.'

In [14]:
search_tool = Tool(
    name='This a general tool for asking quesiton which does not require searching from google',
    func=lambda q: str(conversation.predict(human_input=q)),
    description = """
    Use this tool only when the response does not require a search engine response 
    and can be answered using model internal knowledge
    """,
        return_direct=True,
)

In [15]:
from langchain.chains.conversation.memory import (
    ConversationBufferMemory,
    ConversationBufferWindowMemory,
)

memory = ConversationBufferWindowMemory(
                    memory_key="chat_history", return_messages=True
                )


sarufi_chat_agent = initialize_agent(
    tools=[search_tool],
    llm=llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    handle_parsing_errors=True,
)

In [16]:
sarufi_chat_agent.run(input='What alteranative medicines to treating asthama')



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "There are many alternative medicines that can be used to treat asthma. Some of these include acupuncture, chiropractic care, homeopathy, and herbal medicine."
}

> Finished chain.


'There are many alternative medicines that can be used to treat asthma. Some of these include acupuncture, chiropractic care, homeopathy, and herbal medicine.'

In [21]:
from langchain.utilities import SerpAPIWrapper
name = "Serp Search API"
description = "Use this tool to answer all questions asked by the user"
search = SerpAPIWrapper(
    serpapi_api_key="d0fa4c8a47b66940cde1610ed5c0b7de4ed5fa9a0aeb46f75e3dbbe24ae78c88",
    params={
        "engine": "google_scholar",
        "google_domain": "google.com",
        "gl": "us",
        "hl": "en",
        "num": 3,
        "safe": "active",
    },
)
search_engine_tool = Tool(name=name, description=description, func=search.run)


sarufi_chat_agent = initialize_agent(
    tools=[search_engine_tool, ],
    llm=llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    handle_parsing_errors=True,
)

print(search_engine_tool)

name='Serp Search API' description='Use this tool to answer all questions asked by the user' func=<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google_scholar', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en', 'num': 3, 'safe': 'active'}, serpapi_api_key='d0fa4c8a47b66940cde1610ed5c0b7de4ed5fa9a0aeb46f75e3dbbe24ae78c88', aiosession=None)>


In [22]:
instructions = "You are IMCI assistant, a very powerful health assistant. Answer the questions in a polite and friendly manner. Your responses must be  clear and concise and in the language used to by user in the conversation. You are tasked to be helpful assistant providing guide on how to provide treatment to children in a step wise guideline on Integrated Management of Childhood Illness. Use a search tool provided to find articles related to the question asked whenever a question is asked. Provide links and summary of the articles found, You must provided response as numbered paragraphs. "
query_str= "What are intersting research papers on alternative to paracetamol for treating fever in children"
question = (
    instructions
    + f"""
    [Respond to the following question in a way that is consistent with the context provided
    and feel natural and conversational to human and also follow instruction.] 
    \n\n<<<{query_str}>>>
"""
)

sarufi_chat_agent.run(input=question)



> Entering new AgentExecutor chain...
{
"action": "Final Answer",
"action_input": "I can use the Serp Search API to find interesting research papers on alternative treatments for fever in children."
}

> Finished chain.


'I can use the Serp Search API to find interesting research papers on alternative treatments for fever in children.'

In [19]:
!pip install google-search-results